In [1]:
!pip install ctransformers[cuda]
!pip install --upgrade git+https://github.com/huggingface/transformers

zsh:1: no matches found: ctransformers[cuda]
  Cloning https://github.com/huggingface/transformers to /private/var/folders/bl/kzfk5ts90gj98y9jcv49ynmc0000gn/T/pip-req-build-5ybx7zup
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /private/var/folders/bl/kzfk5ts90gj98y9jcv49ynmc0000gn/T/pip-req-build-5ybx7zup
  Resolved https://github.com/huggingface/transformers to commit d45f47ab7f7c31991bb98a0302ded59ab6adac31
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.39.0.dev0-py3-none-any.whl size=8662893 sha256=6fe08beccbac81965e2994652e460873a2ea8b561506f33232dda833a4015ba8
  Stored in directory: /private/var/folders/bl/kzfk5ts90gj98y9jcv49ynmc0000gn/T/pip-ephem-wheel-cache-k1vrzt_d/wheels/14/a0/7b/8f6b25ba4110aa215fcb8d6aedd6cd4f9b9b6619190999ac2b
Success

In [1]:
from ctransformers import AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/zephyr-7B-alpha-GGUF",
    model_file="zephyr-7b-alpha.Q4_K_M.gguf",
    model_type="mistral",
    gpu_layers=0,
    hf=True
)
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-alpha")

# Pipeline
generator = pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    max_new_tokens=50,
    repetition_penalty=1.1
)

/Users/ayushjain/Development/DisruptionLab/jinship_sp2024/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ayushjain/Development/DisruptionLab/jinship_sp2024/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 22795.13it/s]


In [2]:
prompt = """<|system|>You are a helpful, respectful and honest assistant for labeling topics..</s>
<|user|>
I have a topic that contains the following documents:
[DOCUMENTS]

The topic is described by the following keywords: '[KEYWORDS]'.

Based on the information about the topic above, please create a short label of this topic. Make sure you to only return the label and nothing more.</s>
<|assistant|>"""


In [3]:
from bertopic.representation import TextGeneration
from bertopic import BERTopic

# Text generation with Zephyr
zephyr = TextGeneration(generator, prompt=prompt)
representation_model = {"Zephyr": zephyr}

# Topic Modeling
topic_model = BERTopic(representation_model=representation_model, verbose=True)

In [4]:
from datasets import load_dataset
dataset = load_dataset("valurank/Topic_Classification")["train"]

# Extract abstracts to train on and corresponding titles
descriptions = dataset["article_text"]
topics = dataset["topic"]

In [5]:
topic_model.fit(descriptions[:10])

2024-03-06 23:41:54,296 - BERTopic - Embedding - Transforming documents to embeddings.
Error during conversion: ValueError('Queue is full! Please try again.')
Batches: 100%|██████████| 1/1 [00:00<00:00,  4.27it/s]
2024-03-06 23:41:55,951 - BERTopic - Embedding - Completed ✓
2024-03-06 23:41:55,951 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
Exception in thread Thread-autoconversion:
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/ayushjain/Development/DisruptionLab/jinship_sp2024/.venv/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/Us